# CS 195: Natural Language Processing
## Handling Long-Term Information in Recurrent Neural Networks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_4_LongTermRecurrence.ipynb)

## Reference

SLP: RNNs and LSTMs, Chapter 9 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/9.pdf

Wikipedia article on Gated Recurrent Unit: https://en.wikipedia.org/wiki/Gated_recurrent_unit

Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras by Jason Brownlee: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [2]:
!poetry add datasets keras tensorflow transformers
import sys
!{sys.executable} -m pip install tensorflow

The following packages are already present in the pyproject.toml and will be skipped:

  • datasets
  • keras
  • transformers

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Using version ^2.15.0 for tensorflow

Updating dependencies
Resolving dependencies... (1.0s)

The current project's Python requirement (>=3.10,<4.0) is not compatible with some of the required packages Python requirement:
  - tensorflow-io-gcs-filesystem requires Python >=3.7, <3.12, so it will not be satisfied for Python >=3.12,<4.0
  - tensorflow-io-gcs-filesystem requires Python >=3.7, <3.11, so it will not be satisfied for Python >=3.11,<4.0
  - tensorflow-io-gcs-filesystem requires Python >=3.7, <3.11, so it will not be satisfied for Python >=3.11,<4.0
  - tensorflow-io-gcs-filesystem requires Python >=3.7, <3.12, so it will not be satisfied for Python >=3.12,

## Example Sentence

*The flights the airline was cancelling were full*

Suppose we have generated `The flights the airline`
* `was` is a good next choice
   - `airline` has context for `was` vs. `were`

Suppose we have generated `The flights the airline was cancelling`
* `was`/`were` depends on `flights`
* much more distance information

## The Vanishing Gradient

The *vanishing gradient* is a common problem in deep neural networks

If weights are < 1, they will get smaller and smaller each node they have to pass through - causing them to have little or not effect

Happens during training too - error/loss is propogated backwards through the network proportional to the weights on each edge
* earlier edges in the network are left with little error to use in adjusting weights

<div>
    <img src="images/vanishing_gradient.png">
</div>

image source: https://www.researchgate.net/figure/A-visualization-of-the-vanishing-gradient-problem-using-the-architecture-depicted-in_fig8_277603865

## LSTM

Long Short-Term Memory (LSTM) networks try to address the vanishing gradient through
* removing unneeded information from the context
* adding information likely needed later

<div>
    <img src="images/LSTM_node.png", width=700>
</div>

image_source: SLP Fig. 19.3, https://web.stanford.edu/~jurafsky/slp3/9.pdf

### How does it do this?

* Explicit **context layer** $c_t$
* neural **gates** that control the flow of information through the layer
    - $f$ - the **forget gate** - delete info from context that is no longer needed
    - $g$ - basic extraction of info from previous hidden state
    - $i$ - the **add gate** - select info to add to current context
    - $o$ - the **output gate** - decide what info is needed for current hidden state
    
<div>
    <table>
    <tr>
        <td><img src="images/hadamard_product.png"></td><td style="text-align: left;"><b>Hadamard product:</b> bitwise multiplication</td>
    </tr>
    <tr>
        <td><img src="images/sigmoid.png"></td><td style="text-align: left;"><b>Sigmoid activation:</b> pushes everything to 0 or 1</td>
    </tr>
    <tr>
        <td><img src="images/tanh.png"></td><td style="text-align: left;"><b>Hyperbolic tangent activation:</b> pushes to 0 or 1, more like identity at the origin</td>
    </tr>
    </table>
</div>

Combining sigmoid with ⊙ has the effect of *masking* out information removing some, leaving others

## Gated Recurrent Unit

A **Gated Recurrent Unit** is a popular unit similar to LSTM, except more lightweight
* no output gate
* no context vector

Performance is often similar, but fewer parameters
* faster
* less memory


<div>
    <img src="images/RNN-vs-LSTM-vs-GRU.png", width=700>
</div>

image source: http://dprogrammer.org/rnn-lstm-gru

## Let's work with some data

We'll do something that should be an easier learning problem: text classification with a recurrent network

<div>
    <img src="images/RNN_classification.png" width=700>
</div>

image source: SLP Fig. 9.8, https://web.stanford.edu/~jurafsky/slp3/9.pdf

### IMDB Reviews Dataset

In [3]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("imdb")
dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
#uncomment these to work with a subset of the data
data_subset_text, _, data_subset_label, _ = train_test_split(dataset["train"]["text"],dataset["train"]["label"],train_size=5000) 
train_data_text,  test_data_text, train_data_label, test_data_label = train_test_split(data_subset_text, data_subset_label,test_size = 0.2)

# uncomment these to use the full original data
# train_data_text = dataset["train"]["text"]
# train_data_label = dataset["train"]["label"]
# test_data_text = dataset["test"]["text"]
# test_data_label = dataset["test"]["label"]

In [10]:
#printing out a sample review
print( train_data_text[125] )
print( train_data_label[125] )

This is one of those films the British Lottery Fund wastes its money on. The main problem is a rambling script which gets nowhere. The characters are not interesting, the story is conventional and insipid, the only thing of interest is the location: the city of Genoa (Genova in Italian). Having only a superficial acquaintance with Genoa, I had no idea of the intricate alleyways of its Old Town, and that the city was so interesting. I had thought Genoa was dull. I am delighted to say that I have been proved wrong. So from the travelogue point of view, this film has interest. The film contains one splendid performance, by a little girl named Perla Haney-Jardine. She has already made seven films despite being only 12, so she seems determined upon a career as an actress, and judging by her performance in this film, she should go far, as she is a natural and has a great deal of talent. Colin Firth, a reliable and professional actor, was on hand for the filming and when asked to be earnest, 

### Importing libraries

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SimpleRNN, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

### Preparing the data

In [12]:
vocab_size = 10000
pad_length = 500

tokenizer = Tokenizer(num_words=vocab_size) #only keep the 10000 most common words
tokenizer.fit_on_texts(train_data_text)
tokenized_train_data = tokenizer.texts_to_sequences(train_data_text)
processed_train_data = pad_sequences(tokenized_train_data,maxlen=pad_length, padding='pre')
tokenized_test_data = tokenizer.texts_to_sequences(test_data_text)
processed_test_data = pad_sequences(tokenized_test_data,maxlen=pad_length, padding='pre')

train_target = np.array(train_data_label)
test_target = np.array(test_data_label)

**Important Note:** I originally had tried `padding='post'` which led to bad results
* having a bunch of 0s at the end of a sequence is really bad when you are only sending the last output to the next layer
* in general, we shouldn't be using post-padding with recurrent networks
 - unfortunately, this doesn't seem to be the problem with our encoder-decoder example, but it is still worth going back and fixing if you want to keep working with it

### Defining a simple LSTM-based architecture

Since this is a binary classification problem, we can use a sigmoid activation and `binary_crossentropy` loss.

In [19]:
embedding_size = 50
hidden_layer_size = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=pad_length))
model.add(Dropout(0.2))
# model.add(LSTM(hidden_layer_size))
model.add(SimpleRNN(hidden_layer_size))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 50)           500000    
                                                                 
 dropout_2 (Dropout)         (None, 500, 50)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               15100     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 515201 (1.97 MB)
Trainable params: 515201 (1.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [21]:
model.fit(processed_train_data,
          train_target,
          epochs = 3,
          batch_size = 64,
          validation_data=(processed_test_data,test_target) )

Epoch 1/3
63/63 [==============================] - 6s 90ms/step - loss: 0.6988 - accuracy: 0.5138 - val_loss: 0.6869 - val_accuracy: 0.5420
Epoch 2/3
63/63 [==============================] - 6s 89ms/step - loss: 0.6246 - accuracy: 0.6977 - val_loss: 0.7023 - val_accuracy: 0.5230
Epoch 3/3
63/63 [==============================] - 6s 89ms/step - loss: 0.5971 - accuracy: 0.6867 - val_loss: 0.6697 - val_accuracy: 0.5880


## Exercise

The source I got this code from (https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/ ) included Dropout layers - you can try uncommenting them and see what it does.

It's also equivalent to writing

`model.add(LSTM(hidden_layer_size), dropout=0.2, recurrent_dropout=0.2)`

Do some searching and see what you can find out about what dropout layers do and why people use them. Discuss your findings with your group.

<div style="background: #007acc; color: white; border-radius: 10px; padding: 10px; width: fit-content; max-width: 70%; font-size: 1rem;">
    Does dropout help? No, it was about the same with the current training
</div>

## Exercise

Run an experiment: What is the difference between using `SimpleRNN` and `LSTM` with this data?

## Applied Exploration

Do one of the following:

1. Redo your experiment with another classification dataset. Choose something with more than 2 classes - this will be good practice is understanding the difference you need to make to the model and data prep. Describe your data and results as usual.
    * I also suggest including a GRU layer in your experiment as well: https://keras.io/api/layers/recurrent_layers/gru/

2. Edit the Encoder-Decoder code from last time to use LSTM or GRU.
    * Note that since LSTM returns both a context and hidden state, you will get an output, a hidden state, and context returned from the LSTM layer (instead of just the output and state). It will look something like
    

In [ ]:
encoder_rnn = LSTM(100, return_state=True)
encoder_outputs, state_h, state_c = encoder_rnn(enc_emb)

and you will pass both state_h, state_c as the *context vector* which is the initial state for the decoder. See the source from last time to flesh out the example: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html